In [1]:
# Dependencies
import numpy as np
import pandas as pd

import csv
import os

# Imports the method used for connecting to DBs
from sqlalchemy import create_engine, MetaData

# Imports the methods needed to abstract classes into tables
from sqlalchemy.ext.declarative import declarative_base

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float

In [2]:
# Read the data and cleaning it up
#pd.read_csv(os.path.join("raw_data","city_data.csv"))
summaryDf = pd.read_csv(os.path.join("static","resources","data","summaryData.csv"))
summaryDf['DummyFld'] = "X"

ExistingDevDf = summaryDf[["TSA","Dis_SubDis","LUCategory","EX_OffSQFT","EX_RetSQFT","EX_HotSQFT","EX_InstSQFT","EX_IndusSQFT","EX_NonResGFA","EX_ResSQFT","EX_ResDu"]]
ExistingDevDf = ExistingDevDf.rename(columns = {'EX_OffSQFT': 'Office','EX_RetSQFT': 'Retail' ,'EX_HotSQFT': 'Hotel','EX_InstSQFT': 'Institutional','EX_IndusSQFT': 'Industrial','EX_NonResGFA': 'Nonresidential_GFA','EX_ResSQFT': 'Residential_GFA','EX_ResDu': 'Residential_Units'})
ExistingDevDf['Scenario'] = "Existing Development"


PlanMaxDf = summaryDf[["TSA","Dis_SubDis","LUCategory","MR_OFF_SQFT","MR_RET_SQFT","MR_HOT_SQFT","MR_INS_SQFT","MR_IND_SQFT","MR_NonRes","MR_ResGFA","MR_ResDU"]]
PlanMaxDf = PlanMaxDf.rename(columns = {'MR_OFF_SQFT': 'Office','MR_RET_SQFT': 'Retail' ,'MR_HOT_SQFT': 'Hotel','MR_INS_SQFT': 'Institutional','MR_IND_SQFT': 'Industrial','MR_NonRes': 'Nonresidential_GFA','MR_ResGFA': 'Residential_GFA','MR_ResDU': 'Residential_Units'})
PlanMaxDf['Scenario'] = "Plan Buildout"

ExAppDevDf = summaryDf[["TSA","Dis_SubDis","LUCategory","EARMR_OffSQFT","EARMR_RetSQFT","EARMR_HotSQFT","EARMR_InstSQFT","EARMR_IndusSQFT","EARMR_NonResGFA","EARMR_ResSQFT","EARMR_ResDu"]]
ExAppDevDf = ExAppDevDf.rename(columns = {'EARMR_OffSQFT': 'Office','EARMR_RetSQFT': 'Retail' ,'EARMR_HotSQFT': 'Hotel','EARMR_InstSQFT': 'Institutional','EARMR_IndusSQFT': 'Industrial','EARMR_NonResGFA': 'Nonresidential_GFA','EARMR_ResSQFT': 'Residential_GFA','EARMR_ResDu': 'Residential_Units'})
ExAppDevDf['Scenario'] = "Existing + Approved "

ExURAppDevDf = summaryDf[["TSA","Dis_SubDis","LUCategory","EUARMR_OffSQFT","EUARMR_RetSQFT","EUARMR_HotelSQFT","EUARMR_InstSQFT","EUARMR_IndusSQFT","EUARMR_NonResGFA","EUARMR_ResSQFT","EUARMR_ResDu"]]
ExURAppDevDf = ExURAppDevDf.rename(columns = {'EUARMR_OffSQFT': 'Office','EUARMR_RetSQFT': 'Retail' ,'EUARMR_HotelSQFT': 'Hotel','EUARMR_InstSQFT': 'Institutional','EUARMR_IndusSQFT': 'Industrial','EUARMR_NonResGFA': 'Nonresidential_GFA','EUARMR_ResSQFT': 'Residential_GFA','EUARMR_ResDu': 'Residential_Units'})
ExURAppDevDf['Scenario'] = "Existing + Approved + UnderReview"

ExURAppDevDf.head()



,TSA,Dis_SubDis,LUCategory,Office,Retail,Hotel,Institutional,Industrial,Nonresidential_GFA,Residential_GFA,Residential_Units,Scenario
0,Reston Town Center TSA,Central Sunrise Valley,Government / Institutional,1096572,0,0,0,0,1096572.0,0,0,Existing + Approved + UnderReview
1,Wiehle-Reston East TSA,Reston East,Government / Institutional,0,0,0,0,12599,12599.0,0,0,Existing + Approved + UnderReview
2,Wiehle-Reston East TSA,Wiehle Station North,Government / Institutional,0,0,0,9480,0,9480.0,0,0,Existing + Approved + UnderReview
3,Reston Town Center TSA,Central Sunrise Valley,Industrial,438678,0,0,0,709082,204160.0,0,0,Existing + Approved + UnderReview
4,Herndon TSA,Woodland Park,Mixed Use,2503706,165081,113720,7014,0,2355936.0,1813220,1623,Existing + Approved + UnderReview


In [3]:
# Database creation
import sqlalchemy
# Imports the method used for connecting to DBs
from sqlalchemy import create_engine

# Imports the methods needed to abstract classes into tables
from sqlalchemy.ext.declarative import declarative_base

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float, Text

In [4]:
# Sets an object to utilize the default declarative base in SQL Alchemy
Base = declarative_base()


In [5]:
# Create an engine to a SQLite database file called `RestonDev.sqlite`

engine = create_engine("sqlite:///static/resources/data/RestonDev.sqlite")

In [6]:
# Create a connection to the engine called `conn`
conn = engine.connect()

In [7]:
# Creating four classes - Existing, PlanMax , ExApp and ExURApp
class Existing (Base):
    __tablename__ = 'Existing'

    id = Column(Integer, primary_key=True)
    TSA = Column(String)
    Dis_SubDis = Column(String)
    LUCategory = Column(String)
    Office = Column(Integer)
    Retail = Column(Integer)
    Hotel = Column(Integer)
    Institutional = Column(Integer)
    Industrial = Column(Integer)
    Nonresidential_GFA = Column(Integer)
    Residential_GFA = Column(Integer)
    Residential_Units = Column(Integer)
    Scenario = Column(String)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"
class PlanMax (Base):
    __tablename__ = 'PlanMax'

    id = Column(Integer, primary_key=True)
    TSA = Column(String)
    Dis_SubDis = Column(String)
    LUCategory = Column(String)
    Office = Column(Integer)
    Retail = Column(Integer)
    Hotel = Column(Integer)
    Institutional = Column(Integer)
    Industrial = Column(Integer)
    Nonresidential_GFA = Column(Integer)
    Residential_GFA = Column(Integer)
    Residential_Units = Column(Integer)
    Scenario = Column(String)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"
class ExApp (Base):
    __tablename__ = 'ExApp'

    id = Column(Integer, primary_key=True)
    TSA = Column(String)
    Dis_SubDis = Column(String)
    LUCategory = Column(String)
    Office = Column(Integer)
    Retail = Column(Integer)
    Hotel = Column(Integer)
    Institutional = Column(Integer)
    Industrial = Column(Integer)
    Nonresidential_GFA = Column(Integer)
    Residential_GFA = Column(Integer)
    Residential_Units = Column(Integer)
    Scenario = Column(String)
    def __repr__(self):
        return f"id={self.id}, name={self.name}"
class ExURApp (Base):
    __tablename__ = 'ExURApp'

    id = Column(Integer, primary_key=True)
    TSA = Column(String)
    Dis_SubDis = Column(String)
    LUCategory = Column(String)
    Office = Column(Integer)
    Retail = Column(Integer)
    Hotel = Column(Integer)
    Institutional = Column(Integer)
    Industrial = Column(Integer)
    Nonresidential_GFA = Column(Integer)
    Residential_GFA = Column(Integer)
    Residential_Units = Column(Integer)
    Scenario = Column(String)
    def __repr__(self):
        return f"id={self.id}, name={self.name}"
    


In [8]:
# Use `create_all` to create the above tables in the database
Base.metadata.create_all(engine)

In [9]:
# coverting the dataframes to dictionaries

ExistingData = ExistingDevDf.to_dict(orient='records')
PlanMaxData = PlanMaxDf.to_dict(orient='records')
ExAppData = ExAppDevDf.to_dict(orient='records')
ExURAppData = ExURAppDevDf.to_dict(orient='records')


In [10]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [11]:
# Save the reference to the tables
ExistingTable = sqlalchemy.Table('Existing', metadata, autoload=True)
PlanMaxTable = sqlalchemy.Table('PlanMax', metadata, autoload=True)
ExAppTable = sqlalchemy.Table('ExApp', metadata, autoload=True)
ExURAppTable = sqlalchemy.Table('ExURApp', metadata, autoload=True)

In [12]:
# Insert the tables. Before you do that delete any pre-exsiting tables
conn.execute(ExistingTable.delete())
conn.execute(PlanMaxTable.delete())
conn.execute(ExAppTable.delete())
conn.execute(ExURAppTable.delete())


conn.execute(ExistingTable.insert(), ExistingData)
conn.execute(PlanMaxTable.insert(), PlanMaxData)
conn.execute(ExAppTable.insert(), ExAppData)
conn.execute(ExURAppTable.insert(), ExURAppData)

In [13]:
#Testing 

conn.execute("select * from ExURApp").fetchall()

[(1, 'Reston Town Center TSA', 'Central Sunrise Valley', 'Government / Institutional', 1096572, 0, 0, 0, 0, 1096572, 0, 0, 'Existing + Approved + UnderReview'),
 (2, 'Wiehle-Reston East TSA', 'Reston East', 'Government / Institutional', 0, 0, 0, 0, 12599, 12599, 0, 0, 'Existing + Approved + UnderReview'),
 (3, 'Wiehle-Reston East TSA', 'Wiehle Station North', 'Government / Institutional', 0, 0, 0, 9480, 0, 9480, 0, 0, 'Existing + Approved + UnderReview'),
 (4, 'Reston Town Center TSA', 'Central Sunrise Valley', 'Industrial', 438678, 0, 0, 0, 709082, 204160, 0, 0, 'Existing + Approved + UnderReview'),
 (5, 'Herndon TSA', 'Woodland Park', 'Mixed Use', 2503706, 165081, 113720, 7014, 0, 2355936, 1813220, 1623, 'Existing + Approved + UnderReview'),
 (6, 'Reston Town Center TSA', 'East Fountain Drive', 'Mixed Use', 488683, 263222, 0, 0, 0, 333005, 0, 0, 'Existing + Approved + UnderReview'),
 (7, 'Reston Town Center TSA', 'Old Reston Avenue', 'Mixed Use', 567061, 13946, 0, 0, 0, 436424, 29516